<center><h1>Mathematical Formulation</h1></center>

The economical dispach of thermal generation units can be formulated as an objetive function, which should be minimized considering the limits of each power plant and also the supply of the demanded power. The following system of equations formulate the problem:

$$
\begin{equation}
    \begin{aligned}
       \text{Min } & F_t = \sum_{i=1}^{n_g} F_i(P_i) \\
       \text{s.t}  & \\
                   & \sum_{i=1}^{n_g} P_i = P_D \\
                   & P_i^{min} \leq P_i \leq P_i^{max} \\
    \end{aligned}
\end{equation}
$$

Onde:
 - $F_t$ = total financial cost of the dispach.
 - $F_i$ = financial cost function for the the generator $i$.
 - $P_i$ = power generated by the $i$. 
 - $P_D$ = power demand.
 - $P_i^{min}$ = minimum generation limit of the generator $i$.
 - $P_i^{max}$ = maximum generation limit of the generator $i$;
 - $n_g$ = number of generatio units.

A quadratic function can be used as a proxy of the fuel cost for each generation unit (Zhu, 2015). 

$$
\begin{equation}
    F_i(P_i) = c_iP_i^2 + b_iP_i + a_i
\end{equation}
$$

Bringin a more realistic approach to the economic dispach, a thermal generation unit might also have operative zones (Orero e Irving, 1996), which are directly related to auxiliary systems such as boilers and feed pumps. Such operative zones should be avoided as they bring risk to a safe operation of the plant, hence a discontinuous mathematical formulation rises:

$$
\begin{equation}
        P_i^{min} \leq P_i \leq P_i^{n_1} \\
        P_i^{n_2} \leq P_i \leq P_i^{n_3} \\
        P_i^{n_4} \leq P_i \leq P_i^{max} \\
\end{equation}
$$

---

# Solver

In order to solve the problem the [CVXOPT](https://cvxopt.org/) library will be used. The library offers a complete set of mathematical solvers implemented.

### Loading `cvxopt`

In [1]:
from cvxopt import matrix, solvers

solvers.options['show_progress'] = False
# solvers.options['refinement'] = 2
solvers.options['maxiters'] = 15

### Loading data

From now on we can use the `systems` module and initialize a `PowerSystem`, which should provide all of the necessary data for the solver. 

In [2]:
from acopoweropt import system

PSystem = system.PowerSystem(name='s15')

### Initializing variables

In [11]:
from cvxopt import matrix, solvers

import pandas as pd
import numpy as np

data = PSystem.sample_operation()
demand = PSystem.demand

def solve(demand:float, data: pd.DataFrame):
    
    demand = np.array([demand], dtype="double")
    
    # Equation parameters cP^2 + bP + a
    a = data.a.sum()
    b = data.b.to_numpy(dtype="double")
    c = data.c.to_numpy(dtype="double")
    
    # CVXOPT needs a system of equations:
    Pmin = data.Pmin.to_numpy(dtype="double")
    Pmax = data.Pmax.to_numpy(dtype="double")

    P = matrix(2 * (c[..., None] * np.eye(data.shape[0])))
    q = matrix(b)

    G_min = -1 * np.eye(smpl_opr.shape[0])
    G_max = np.eye(smpl_opr.shape[0])
    G = matrix(np.concatenate((G_min,G_max)))
    h = matrix(np.concatenate((-1 * Pmin,Pmax)))

    A = matrix(np.ones(smpl_opr.shape[0]), (1, c.shape[0]))
    b = matrix(demand)
    
    solution = solvers.qp(P, q, G, h, A, b)
    return solution

In [13]:
sol = solve(demand=demand, data=data)

     pcost       dcost       gap    pres   dres
 0:  2.7938e+04 -1.0679e+05  3e+05  1e-01  4e-15
 1:  2.7815e+04 -3.1616e+03  3e+04  5e-04  4e-14
 2:  2.7796e+04  2.6426e+04  1e+03  2e-05  2e-15
 3:  2.7418e+04  2.7039e+04  4e+02  2e-06  1e-16
 4:  2.7341e+04  2.7281e+04  6e+01  2e-07  6e-17
 5:  2.7322e+04  2.7319e+04  3e+00  4e-09  9e-17
 6:  2.7321e+04  2.7321e+04  4e-02  5e-11  1e-16
 7:  2.7321e+04  2.7321e+04  4e-04  5e-13  8e-17
Optimal solution found.


In [14]:
type(sol)

dict